<h2>Dependecy Imports</h2>

In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle
from joblib import dump, load
from sklearn.model_selection import train_test_split


<h2>Import data</h2>

In [2]:
real = pd.read_csv("./data/True.csv")
fake = pd.read_csv("./data/Fake.csv")

# add label "0" for fake "1" for real

real["label"] = 1
fake["label"] = 0

# combine and shuffle data

data = pd.concat([fake, real], axis=0)
data = data.sample(frac=1)
data.reset_index(inplace=True) 
data.drop(["index"], axis=1, inplace=True) 
data

,title,text,subject,date,label
0,LOVE HIM OR HATE HIM…HERE ARE 7 WAYS TRUMP HEL...,"The GOP doesn t know it yet, but they need Tru...",politics,"Jul 25, 2015",0
1,Brazil detains Italian fugitive Battisti leavi...,BRASILIA (Reuters) - Brazilian highway police ...,worldnews,"October 4, 2017",1
2,Hacking Electronic Voting Machines is Easy,21st Century Wire says It s the big dirty sec...,Middle-east,"November 6, 2016",0
3,DEMOCRAT LAWMAKER Puts Forth Bill Requiring Wi...,Leave it to a Democrat to waste everyone s tim...,politics,"Feb 17, 2016",0
4,WAS MICHELLE OBAMA In On Beyonce’s Cop-Hating ...,In an interview that is a tradition before the...,politics,"Feb 8, 2016",0
...,...,...,...,...,...
44893,Trump Considering Incredibly Ironic Choice Fo...,Another day in the Trump Administration means ...,News,"January 26, 2017",0
44894,Chicago Police Dept. Points Out TWO Huge Flaw...,Trump s wild lies continue unabated as he cont...,News,"August 25, 2016",0
44895,Four Serbs get suspended sentences for 2008 at...,BELGRADE (Reuters) - Four Serbian men got susp...,worldnews,"November 14, 2017",1
44896,PICTURE OF Cop Walking Son To School On Day He...,"So what s worse, Barack Obama remaining silent...",left-news,"Jul 29, 2017",0


<h2>Cleaning and Preprocessing</h2>

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://S+ | www\.\S+', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub('\[.*?\]', '', text)
    words=[]
    for i in text:
        if i not in string.punctuation:
            words.append(i)
    return ''.join(words)
    
data['title'] = data['title'].apply(clean_text)
data['text'] = data['text'].apply(clean_text)

data.shape


(44898, 5)

<h2>Test train split</h2>

In [4]:
X,y = data["text"], data["label"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [5]:
# count unique words
word_count = {}
word_index = 1
for text in data["text"]:
    for word in text.split():
        if word not in word_count:
            word_count[word] = word_index
            word_index += 1

len(word_count)            

244200

<h2>Preprocess Data</h2>

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Assuming 'X_train' and 'X_test' contain the text data for training and testing respectively

# Define parameters
vocab_size = len(word_count)  # Adjust this based on your dataset and vocabulary size requirements
max_length = 100    # Adjust this based on the maximum sequence length you want to use

# Tokenize text data
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

with open('./server/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Convert text data to sequences of integers
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post', truncating='post')

# Now, X_train_padded and X_test_padded contain the tokenized and padded sequences ready for training and testing


<h2>Compile LSTM</h2>

In [7]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# Define vocabulary size and maximum sequence length
vocab_size = len(word_count)
max_length = 100

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          24420000  
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 24462305 (93.32 MB)
Trainable params: 24462305 (93.32 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [8]:
#train model
history = model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_data=(X_test_padded, y_test))

Epoch 1/5
1123/1123 [==============================] - 627s 557ms/step - loss: 0.1648 - accuracy: 0.9494 - val_loss: 0.1156 - val_accuracy: 0.9732
Epoch 2/5
1123/1123 [==============================] - 138s 123ms/step - loss: 0.1332 - accuracy: 0.9619 - val_loss: 0.0721 - val_accuracy: 0.9834
Epoch 3/5
1123/1123 [==============================] - 107s 96ms/step - loss: 0.0673 - accuracy: 0.9847 - val_loss: 0.0554 - val_accuracy: 0.9888
Epoch 4/5
1123/1123 [==============================] - 112s 100ms/step - loss: 0.0362 - accuracy: 0.9920 - val_loss: 0.0497 - val_accuracy: 0.9889
Epoch 5/5
1123/1123 [==============================] - 114s 101ms/step - loss: 0.0377 - accuracy: 0.9906 - val_loss: 0.0337 - val_accuracy: 0.9918


In [33]:
#Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test Accuracy:", accuracy)

#save model
model.save("model.h5")


281/281 [==============================] - 2s 8ms/step - loss: 0.0246 - accuracy: 0.9941
Test Accuracy: 0.9940980076789856


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
#sample prediction
sample = data["text"][13]
label = data["label"][13]

print(label)
new_text_sequence = tokenizer.texts_to_sequences([sample])  # Assuming 'tokenizer' is the tokenizer used during training
new_text_padded = pad_sequences(new_text_sequence, maxlen=max_length)  # Assuming 'max_length' is the maximum sequence length used during training

# Make predictions
predictions = model.predict(new_text_padded)
predictions

1
1/1 [==============================] - 0s 14ms/step


array([[0.8873234]], dtype=float32)